---
title: "Regression & Interpretability: When Linear Models Go Sideways"
author: "Rohit Kumar, Tarkanpet"
format:
  html:
    theme: flatly
    toc: true
    toc-depth: 2
    smooth-scroll: true
execute:
  echo: false
  warning: false
  message: false
---


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

# Generate the data directly (same as in the starter file)
observDF = pd.DataFrame({
    "Stress":       [0, 0, 0, 1, 1, 2, 2, 2, 8, 8, 8, 12, 12, 12, 12],
    "StressSurvey": [0, 0, 0, 3, 3, 3, 6, 6, 9, 9, 9, 12, 12, 12, 12],
    "Time":         [0, 1, 1, 1, 1, 1, 2, 2, 2.1, 2.2, 2.2, 2.2, 2.2, 2.2, 2.2],
    "Anxiety":      [0, 0.1, 0.1, 1.1, 1.1, 1.1, 1.1, 2.2, 2.2, 2.2, 8.2, 8.21, 12.22, 12.22, 12.22],
})

observDF = observDF[["Stress", "StressSurvey", "Time", "Anxiety"]]

# Global plot style – darkgrid baseline
sns.set_theme(style="darkgrid", font_scale=1.15)
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

# Helper styling for dark axes so labels / ticks are clearly visible
def style_dark(ax):
    ax.set_facecolor("#020617")
    ax.figure.set_facecolor("#020617")
    ax.tick_params(colors="white", labelsize=11)
    for spine in ax.spines.values():
        spine.set_color("white")
        spine.set_linewidth(1.0)
    ax.xaxis.label.set_color("white")
    ax.yaxis.label.set_color("white")
    ax.title.set_color("white")

def style_legend_dark(leg):
    if leg is None:
        return
    for text in leg.get_texts():
        text.set_color("white")
    if leg.get_title() is not None:
        leg.get_title().set_color("white")
    frame = leg.get_frame()
    frame.set_edgecolor("white")
    frame.set_facecolor("#020617")

# Core models
m_stressSurvey = smf.ols("Anxiety ~ StressSurvey", data=observDF).fit()
m_time_only    = smf.ols("Anxiety ~ Time", data=observDF).fit()
m_survey_time  = smf.ols("Anxiety ~ StressSurvey + Time", data=observDF).fit()
m_true_stress  = smf.ols("Anxiety ~ Stress + Time", data=observDF).fit()

# Subset where Stress is low / moderate
subset = observDF[observDF["Stress"] <= 2]
m_subset = smf.ols("Anxiety ~ StressSurvey + Time", data=subset).fit()

def pretty_coef_table(model, title):
    """Return a styled coefficient table for a statsmodels OLS result."""
    df = pd.DataFrame({
        "Term": model.params.index,
        "Estimate": model.params.values,
        "Std. Error": model.bse.values,
        "t": model.tvalues.values,
        "p-value": model.pvalues.values,
    })

    df["p-value"] = df["p-value"].round(4)

    styled = (
        df.style
          .format({
              "Estimate": "{:.3f}",
              "Std. Error": "{:.3f}",
              "t": "{:.2f}",
          })
          .set_caption(title)
          .background_gradient(axis=None, cmap="PuRd")
          .set_table_attributes(
              'class="table table-sm table-hover" style="width:100%; table-layout:fixed;"'
          )
          .hide(axis="index")
    )
    return styled

```{=html}
<style>
/* keep figures and tables inside the content width, no horizontal scroll */
.quarto-figure, .cell-output-display, table {
  max-width: 100%;
}

/* center figures */
.quarto-figure img {
  display: block;
  margin-left: auto;
  margin-right: auto;
}

/* make pandas styled tables look like cards */
table.dataframe {
  border-radius: 12px;
  overflow: hidden;
}
</style>
```

## Data and true model

The variables in the dataset:

We’ll work with a small dataset of students that has four variables:

- **Anxiety** – outcome from an fMRI-based anxiety measure  
- **Stress** – “true” stress level from a blood test  
- **StressSurvey** – survey-based proxy for stress  
- **Time** – hours spent on social media in the last 24 hours  

By construction, the true relationship in the population is

\[
Anxiety = Stress + 0.1 \times Time
\]

So in the correctly specified model with `Stress` and `Time` as predictors, the true coefficients are:

- \( \beta_1 = 1 \) for Stress  
- \( \beta_2 = 0.1 \) for Time  

`StressSurvey` is designed to track Stress, but with a non-linear distortion.


In [ ]:
#| fig-cap: How the survey score tracks true stress.
#| out-width: 100%
#| fig-align: center

fig, ax = plt.subplots()

sns.lineplot(
    data=observDF.sort_values("Stress"),
    x="Stress", y="StressSurvey",
    marker="o", linewidth=2.8, ax=ax,
    color="#38bdf8"
)
for _, row in observDF.drop_duplicates("Stress").iterrows():
    ax.annotate(int(row["Stress"]),
                (row["Stress"], row["StressSurvey"]),
                textcoords="offset points", xytext=(0,7),
                ha="center", fontsize=9, color="white")

ax.set_xlabel("True Stress (blood test)")
ax.set_ylabel("StressSurvey")
ax.set_title("Survey Stress vs. True Stress")
style_dark(ax)
plt.tight_layout()
plt.show()

The survey score clearly increases with true Stress, but the steps are uneven, especially at higher stress levels. That warped scale is what will cause trouble for the regression models later on.


---

## Q1 – Bivariate regression: Anxiety on StressSurvey

Model:

\[
Anxiety = \alpha_0 + \alpha_1 \times StressSurvey + \varepsilon.
\]

In [ ]:
pretty_coef_table(m_stressSurvey, "Bivariate regression: Anxiety ~ StressSurvey")

From this fit, the slope on **StressSurvey** is positive and statistically significant, and the \(R^2\) value shows that the model explains a large share of the variation in Anxiety.

Interpreted literally, the model says that each one-unit increase in the survey score is associated with roughly one additional unit of Anxiety, on average. That sounds reasonable, but it is not quite what we want. The true relationship is defined in terms of **Stress**, not StressSurvey, and the survey scale is warped. So even though the fit looks strong, the coefficient on StressSurvey does not directly tell us the causal effect of true stress.


In [ ]:
#| fig-cap: Anxiety vs. StressSurvey with fitted bivariate regression line.
#| out-width: 100%
#| fig-align: center

fig, ax = plt.subplots()

sns.scatterplot(
    data=observDF,
    x="StressSurvey", y="Anxiety",
    s=150, edgecolor="white", linewidth=0.7,
    hue="Stress", palette="viridis", ax=ax, alpha=0.95
)

x_vals = np.linspace(observDF["StressSurvey"].min(),
                     observDF["StressSurvey"].max(), 200)
y_hat = (m_stressSurvey.params["Intercept"] +
         m_stressSurvey.params["StressSurvey"] * x_vals)
ax.plot(x_vals, y_hat, color="#f97316", linewidth=3.2, linestyle="-", label="Fitted line")

ax.set_xlabel("StressSurvey")
ax.set_ylabel("Anxiety")
ax.set_title("Anxiety vs. StressSurvey")
leg = ax.legend(title="True Stress", loc="upper left", frameon=True)
style_dark(ax)
style_legend_dark(leg)
plt.tight_layout()
plt.show()

The scatterplot has a gentle curve, and the straight regression line cuts across it. That already hints that forcing a single constant slope onto this relationship is an oversimplification of how Anxiety responds to Stress in this dataset.

---

## Q2 – Bivariate regression: Anxiety on Time

Now regress Anxiety on Time only:

\[
Anxiety = \gamma_0 + \gamma_1 \times Time + \varepsilon.
\]

In [ ]:
pretty_coef_table(m_time_only, "Bivariate regression: Anxiety ~ Time")

From this fit, the slope on **Time** is positive and statistically significant. However, the estimated effect is much larger than the true marginal effect of 0.1 that we know is built into the data, and the \(R^2\) value is noticeably lower than in the StressSurvey model. Time alone is clearly picking up something about Anxiety, but it is not telling the full story.


In [ ]:
#| fig-cap: Anxiety vs. Time on social media with bivariate regression line.
#| out-width: 100%
#| fig-align: center

fig, ax = plt.subplots()

sns.scatterplot(
    data=observDF,
    x="Time", y="Anxiety",
    s=150, edgecolor="white", linewidth=0.7,
    hue="StressSurvey", palette="magma", ax=ax, alpha=0.95
)

x_vals = np.linspace(observDF["Time"].min(),
                     observDF["Time"].max(), 200)
y_hat = (m_time_only.params["Intercept"] +
         m_time_only.params["Time"] * x_vals)
ax.plot(x_vals, y_hat, color="#22c55e", linewidth=3.2, label="Fitted line")

ax.set_xlabel("Time on Social Media (hours)")
ax.set_ylabel("Anxiety")
ax.set_title("Anxiety vs. Time on Social Media")
leg = ax.legend(title="StressSurvey", loc="upper left", frameon=True)
style_dark(ax)
style_legend_dark(leg)
plt.tight_layout()
plt.show()

Visually, it looks like a clean positive relationship: students who spend more time on social media also tend to have higher Anxiety. But a lot of that pattern comes from the fact that higher-stress students also tend to appear at higher Time values. In other words, Time is acting as a proxy for Stress when Stress is not included in the model.


---

## Q3 – Multiple regression with StressSurvey and Time

Next, add both predictors:

\[
Anxiety = \delta_0 + \delta_1 \times StressSurvey + \delta_2 \times Time + \varepsilon.
\]

In [ ]:
pretty_coef_table(m_survey_time, "Multiple regression: Anxiety ~ StressSurvey + Time")

The key results are:

- StressSurvey coefficient \( \hat\delta_1 \approx 1.36 \), positive and statistically significant  
- Time coefficient \( \hat\delta_2 \approx -3.46 \), negative but not statistically significant at the 5% level  
- \( R^2 \) is high, which on its own makes the model look strong  

The striking part is the **sign flip** on the Time coefficient. In the true data-generating process, more Time slightly *increases* Anxiety, but once we “control” for StressSurvey, the fitted model suggests that more Time is associated with *lower* Anxiety.


In [ ]:
#| fig-cap: True vs. estimated coefficients (using StressSurvey + Time) – horizontal bar view.
#| out-width: 100%
#| fig-align: center

true_vals = pd.DataFrame({
    "Model": ["True model"],
    "Stress-like": [1.0],
    "Time": [0.1]
})

survey_vals = pd.DataFrame({
    "Model": ["StressSurvey + Time"],
    "Stress-like": [m_survey_time.params["StressSurvey"]],
    "Time": [m_survey_time.params["Time"]]
})

coef_df = (
    pd.concat([true_vals, survey_vals], ignore_index=True)
      .melt(id_vars="Model", var_name="Coefficient", value_name="Estimate")
)

fig, ax = plt.subplots()

palette = {"True model": "#22c55e", "StressSurvey + Time": "#f97316"}

sns.barplot(
    data=coef_df,
    x="Coefficient", y="Estimate",
    hue="Model", ax=ax, palette=palette, edgecolor="white"
)
ax.axhline(0, color="white", linewidth=1)
ax.set_xlabel("Coefficient")
ax.set_ylabel("Estimate")
ax.set_title("True Coefficients vs. Model Using StressSurvey")
leg = ax.legend(loc="upper left", frameon=True)
style_dark(ax)
style_legend_dark(leg)

# annotate values at bar ends
for p in ax.patches:
    width = p.get_width()
    ax.text(width + np.sign(width)*0.05, 
            p.get_y() + p.get_height()/2,
            f"{width:.2f}",
            va="center", ha="left" if width > 0 else "right",
            color="white", fontsize=9)

plt.tight_layout()
plt.show()

Even with a high \(R^2\) and at least one very small p-value, this model tells the opposite story about social media time compared to the ground truth. The non-linear survey proxy is bending the regression in a misleading direction.



---

## Q4 – Multiple regression with true Stress and Time

Now fit the model that uses the underlying Stress measure instead of the proxy:

\[
Anxiety = \beta_0 + \beta_1 \times Stress + \beta_2 \times Time + \varepsilon.
\]

In [ ]:
pretty_coef_table(m_true_stress, "Multiple regression: Anxiety ~ Stress + Time (true model)")

Here the estimates from this model are:

- Stress coefficient \( \hat\beta_1 \approx 0.97 \), which is close to the true value of 1.0  
- Time coefficient \( \hat\beta_2 \approx -0.94 \), which does **not** match the mild positive effect of 0.1 that we know is built into the data  
- The overall \(R^2\) is still high, so the model appears to fit well  

This shows that even when we use a more direct measure of stress, a small sample and the particular design of the data can leave the estimated Time effect unstable. The model with true Stress is clearly closer to the data-generating story, but it still reminds us not to over-interpret individual coefficients just because they appear in a neat summary table.

In [ ]:
#| fig-cap: How specification changes the estimated coefficients – true Stress vs survey proxy.
#| out-width: 100%
#| fig-align: center

comp = pd.DataFrame({
    "Model": ["StressSurvey + Time", "StressSurvey + Time",
              "Stress + Time", "Stress + Time"],
    "Coefficient": ["Time", "Stress-like", "Time", "Stress-like"],
    "Estimate": [m_survey_time.params["Time"],
                 m_survey_time.params["StressSurvey"],
                 m_true_stress.params["Time"],
                 m_true_stress.params["Stress"]]
})

fig, ax = plt.subplots()

palette2 = {"StressSurvey + Time": "#f97316", "Stress + Time": "#22c55e"}

sns.barplot(
    data=comp,
    y="Coefficient", x="Estimate",
    hue="Model", ax=ax, palette=palette2, orient="h", edgecolor="white"
)
ax.axvline(0, color="white", linewidth=1)
ax.set_xlabel("Estimate")
ax.set_ylabel("Coefficient")
ax.set_title("How Specification Changes the Story")
leg = ax.legend(loc="lower right", frameon=True)
style_dark(ax)
style_legend_dark(leg)

for p in ax.patches:
    width = p.get_width()
    ax.text(width + np.sign(width)*0.05, 
            p.get_y() + p.get_height()/2,
            f"{width:.2f}",
            va="center", ha="left" if width > 0 else "right",
            color="white", fontsize=9)

plt.tight_layout()
plt.show()

The model that uses StressSurvey gives a large negative Time effect, while the model with true Stress produces a Stress coefficient that is close to the truth and a Time coefficient that is much less extreme but still imprecise.



---

## Q5 – How headlines could mislead people

If the StressSurvey + Time model were reported in the news, a headline might read:

> “After accounting for stress, more time on social media is linked to lower anxiety.”

Everything in the regression table would look legitimate: high \(R^2\), at least one significant coefficient, and reasonable standard errors. Parents reading that headline might feel reassured, and social-media companies would be happy to promote it.

If the model with true Stress were reported instead, a more realistic headline could be:

> “More time on social media is associated with higher anxiety, even after controlling for stress.”

Both models are estimated on exactly the same data but tell opposite stories. The difference comes entirely from how stress is measured and modeled.


---

## Q6 – Subset analysis: focusing on a more linear region

The main distortion comes from how the survey compresses the high-stress range. To see what happens in a region where the relationship is closer to linear, restrict the sample to


\[
\text{Stress} \le 2
\]


where `StressSurvey` is essentially three times Stress.

In [ ]:
pretty_coef_table(m_subset, "Subset regression (Stress ≤ 2): Anxiety ~ StressSurvey + Time")

On this subset the results become:

- StressSurvey coefficient \(\hat\beta_1 \approx 0.29\) (p \(\approx 0.038\))  
- Time coefficient \(\hat\beta_2 \approx -0.09\), small in magnitude and not statistically significant (p \(\approx 0.83\))  
- The model’s \(R^2\) remains high, but not perfect


In this region, `StressSurvey ≈ 3 × Stress`, so a slope around 0.29 on the survey corresponds to just under 0.9 per unit of true Stress. That is much closer to the intended value of 1 than what we saw before. The Time effect is now very small and statistically indistinguishable from zero, which is much closer to the true mild effect of 0.1 than the earlier large negative estimate.

In [ ]:
#| fig-cap: Highlighting the low-stress region where the survey behaves more linearly.
#| out-width: 100%
#| fig-align: center

fig, ax = plt.subplots()

sns.scatterplot(
    data=observDF,
    x="StressSurvey", y="Anxiety",
    color="#4b5563", s=90, ax=ax, label="All data"
)

sns.scatterplot(
    data=subset,
    x="StressSurvey", y="Anxiety",
    s=160, edgecolor="white", linewidth=0.9,
    color="#f97316", ax=ax, label="Subset: Stress ≤ 2"
)

x_vals = np.linspace(subset["StressSurvey"].min(),
                     subset["StressSurvey"].max(), 100)
y_hat = (m_subset.params["Intercept"] +
         m_subset.params["StressSurvey"] * x_vals +
         m_subset.params["Time"] * subset["Time"].mean())
ax.plot(x_vals, y_hat, color="#22c55e", linewidth=3.2,
        label="Subset fit (holding Time at mean)")

ax.set_xlabel("StressSurvey")
ax.set_ylabel("Anxiety")
ax.set_title("Focusing on the Region Where the Proxy is Nearly Linear")
leg = ax.legend(loc="upper left", frameon=True)
style_dark(ax)
style_legend_dark(leg)
plt.tight_layout()
plt.show()

When the proxy behaves almost linearly, the regression has a much easier time recovering a sensible relationship between StressSurvey and Anxiety. Once the strongly non-linear part of the scale is included, the slopes swing to unrealistic values, including sign flips.


---

## Takeaways

- A high \(R^2\) and statistically significant coefficients do **not** guarantee that the slopes tell a sensible story.  
- Using a non-linear proxy (like StressSurvey) in place of a more direct measure (Stress) can completely change, or even invert, the apparent effect of another variable (Time).  
- Visual checks, thinking carefully about how variables are measured, and exploring regions where relationships are closer to linear are essential before trusting a linear regression model for interpretation.
